<a href="https://colab.research.google.com/github/nagswarnaa/MachineLearning/blob/main/ass4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
class Layer:
    def __init__(object):
        object.ip = None
        object.op = None

    def forward(object, data_ip):
        raise NotImplementedError

    def backward(object, error_op, alpha):
        raise NotImplementedError

##Implementing Linear Layer


In [88]:

import numpy as np

class LinearLayer(Layer):

    def __init__(object, len_ip, len_op):
        object.wghts = np.random.rand(len_ip, len_op) - 0.5
        object.bs = np.random.rand(1, len_op) - 0.5

    def forward(object, ip_data):
        object.ip = ip_data
        object.op = np.dot(object.ip, object.wghts) + object.bs
        return object.op

    def backward(object, op_error, learning_rate):
        ip_error = np.dot(op_error, object.wghts.T)
        wghts_error = np.dot(object.ip.T, op_error)
        object.wghts =object.wghts- (learning_rate * wghts_error)
        object.bs =object.bs- (learning_rate * op_error)
        return ip_error

##Tangent Activation Class


In [89]:
class TangentActivation(Layer):
    
    def forward(object, ip_data):
        object.ip = ip_data
        object.op = np.tanh(object.ip)
        return object.op

    def backward(object, op_error, learning_rate):
        return (1-np.tanh(object.ip)**2 )* op_error

##Sigmoid Activation Class

In [91]:
class SigmoidActivation(Layer):
    
    def forward(object, ip_data):
        object.ip = ip_data
        object.op =  (1/(1 + np.exp(-object.ip)))
        return object.op

    def backward(object, op_error, learning_rate):
        return ( np.exp(-object.ip)/(1+np.exp(-object.ip)**2))* op_error

##SoftMax Activation Class

In [92]:
class SoftMaxActivation(Layer):
    
    def forward(object, ip_data):
        object.ip = ip_data
        object.op =  (np.exp(ip_data - np.max(ip_data)) / np.exp(ip_data - np.max(ip_data)).sum())
        return object.op

    def backward(object, op_error, learning_rate):
        return ( np.exp(-object.ip)/(1+np.exp(-object.ip)**2))* op_error

In [93]:
import numpy as np

def cross_entropy_loss(pred,target):
    return -target * np.log(pred)

def cross_entropy_loss_grad(pred,target):
        return target - pred

In [94]:
class Sequential:
    def __init__(object):
        object.layers = []

    def appendLayer(object, layer):
        object.layers.append(layer)

    #Predicting Using Forward Propagation
    def predict(object, ip_data):
        result = []
        for i in range(len(ip_data)):
            op = ip_data[i]
            for layer in object.layers:
                op = layer.forward(op)
            result.append(op)

        return result

    #Training Neural Network
    def fit(object, x_train, y_train, epochs, learning_rate):
        for i in range(epochs):
            err = 0
            for j in range(len(x_train)):
                # forward propagation
                op = x_train[j]
                for layer in object.layers:
                    op = layer.forward(op)

                # backward propagation
                error = cross_entropy_loss_grad(y_train[j], op)
                for layer in reversed(object.layers):
                    error = layer.backward(error, learning_rate)

            

In [95]:
import numpy as np



# Training Data 
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# Using Sigmoid function

#Building Network
nnworkSigm = Sequential()
nnworkSigm.appendLayer(LinearLayer(2, 2))
nnworkSigm.appendLayer(SigmoidActivation())
nnworkSigm.appendLayer(LinearLayer(2, 1))
nnworkSigm.appendLayer(SigmoidActivation())

#Fitting Network
nnworkSigm.fit(x_train, y_train, epochs=1000, learning_rate=0.1)
out = nnworkSigm.predict(x_train)
print(out)


# Using Tan function

#Building Network
nnworkTan = Sequential()
nnworkTan.appendLayer(LinearLayer(2, 3))
nnworkTan.appendLayer(SigmoidActivation())
nnworkTan.appendLayer(LinearLayer(3, 1))
nnworkTan.appendLayer(SigmoidActivation())

#Fitting Network
nnworkTan.fit(x_train, y_train, epochs=1000, learning_rate=0.1)
out = nnworkTan.predict(x_train)
print(out)


[array([[0.5012875]]), array([[0.50001964]]), array([[0.50016406]]), array([[0.49892521]])]
[array([[0.49865008]]), array([[0.50515846]]), array([[0.49580685]]), array([[0.50232822]])]


In [107]:
import numpy as np
from keras.utils import to_categorical
from keras.datasets import mnist



(X_train, y_train), (X_test, y_test) = mnist.load_data()



X_train = X_train.reshape(X_train.shape[0], 1, 28*28)
X_train = X_train.astype('float32')
X_train /= 255


X_test = X_test.reshape(X_test.shape[0], 1, 28*28)
X_test = X_test.astype('float32')
X_test /= 255

y_train = to_categorical(y_train)

inp_layer=784;
hidden1=128;
hidden2=64;
oup_layer=10;


mnistNetwork = Sequential()
mnistNetwork.appendLayer(LinearLayer(inp_layer,hidden1))
mnistNetwork.appendLayer(SigmoidActivation())
mnistNetwork.appendLayer(LinearLayer(hidden1,hidden2))
mnistNetwork.appendLayer(SigmoidActivation())
mnistNetwork.appendLayer(LinearLayer(hidden2,oup_layer))
mnistNetwork.appendLayer(SigmoidActivation())

mnistNetwork.fit(X_train[0:1000], y_train[0:1000], epochs=2, learning_rate=0.1)

out = mnistNetwork.predict(X_test[0:3])
for i in range(3):
  print(np.argmax(out[i]))

print(y_test[0:3])







7
3
1
[7 2 1]


In [14]:
temp = []
for i in range(len(y_train)):
  temp.append(to_categorical(y_train[i], num_classes=10))

y_train = np.array(temp)

temp = []
for i in range(len(y_test)):
  temp.append(to_categorical(y_test[i], num_classes=10))  
   
y_test = np.array(temp)